In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2,RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

In [2]:
df = pd.read_csv("train.csv")
new_cols = ['C' + str(i) for i in range(1, 14 + 1)] + ['X' + str(i) for i in range(1, 19 + 2)]
new_cols[-1] = 'Y'
df.columns = new_cols
# -1 = not happening (negative), 1 = happening (positive)
#df['Y'].replace({-1 : 0}, inplace=True, regex=True)

df.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,X11,X12,X13,X14,X15,X16,X17,X18,X19,Y
0,V1,V1,V1,V1,V1,V1,V2,V1,V3,V5,...,0.352,0.073,-0.092,1.098,0.034,1.160,0.401,0.037,0.216,1
1,V1,V1,V1,V1,V1,V1,V1,V1,V2,V2,...,-1.933,-0.536,-0.092,1.098,0.034,0.716,0.401,0.724,0.216,-1
2,V1,V1,V1,V1,V1,V1,V1,V1,V2,V2,...,-0.762,-0.536,-0.092,1.098,0.034,0.716,0.401,0.712,0.216,-1
3,V1,V1,V1,V1,V1,V1,V1,V1,V1,V1,...,-0.505,-0.536,-0.092,0.182,0.034,0.716,0.401,0.393,0.216,1
4,V1,V1,V1,V1,V1,V1,V1,V1,V1,V1,...,0.409,-0.536,-0.092,0.731,0.034,0.716,0.401,0.724,0.216,1


In [3]:
features = df.loc[:, df.columns != 'Y']
numerical_features = df.select_dtypes(include=['float64']).columns
categorical_features = df.select_dtypes(include=['object']).columns
target = df['Y']

In [4]:
from sklearn.impute import SimpleImputer
numerical_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
numerical_imputer.fit(df[numerical_features])
df[numerical_features] = numerical_imputer.transform(df[numerical_features])

categorical_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
categorical_imputer.fit(df[categorical_features])
df[categorical_features] = categorical_imputer.transform(df[categorical_features])

encoded_df = pd.get_dummies(df)
encoded_features = encoded_df.loc[:, encoded_df.columns != 'Y']

In [5]:
encoded_df.to_csv("export.csv", index=False)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(encoded_df, target, test_size=0.2)
X_train

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,C12_V1,C12_V2,C12_V3,C13_V1,C13_V2,C13_V3,C13_V4,C13_V5,C14_V1,C14_V2
84,-0.00400,-0.862,0.348000,0.110,-0.458,2.717,-1.196,1.215,-1.592,-0.924,...,1,0,0,1,0,0,0,0,0,1
186,1.37800,0.751,1.312000,-0.047,0.706,1.425,-0.099,1.931,1.754,-1.445,...,1,0,0,0,1,0,0,0,0,1
225,-0.28000,0.130,0.823000,1.127,0.189,-0.513,0.997,-0.844,-0.375,-0.241,...,0,1,0,1,0,0,0,0,1,0
308,-0.55700,0.379,-0.127000,5.352,-0.199,-0.513,0.120,0.069,-2.505,0.110,...,0,1,0,1,0,0,0,0,0,1
238,-1.66300,0.379,0.823000,0.736,0.189,0.456,-0.099,-0.844,0.842,0.784,...,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,-0.83300,-0.118,0.007324,0.110,2.128,-0.513,-0.538,-0.844,1.602,1.234,...,0,0,1,0,1,0,0,0,0,1
277,-0.00400,-1.235,-1.092000,-0.282,-0.070,0.133,-1.196,1.288,-0.071,-1.284,...,1,0,0,1,0,0,0,0,1,0
293,-1.44100,-1.111,-0.617000,0.501,-1.104,-0.513,1.873,-0.844,-1.440,1.485,...,0,1,0,1,0,0,0,0,0,1
79,-0.55700,-0.242,-0.617000,-0.047,-0.587,1.425,0.339,-0.844,0.537,1.234,...,1,0,0,1,0,0,0,0,1,0


In [22]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [29]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#classifier = SVC(gamma='auto')
#classifier = BaggingClassifier(n_estimators=1000, base_estimator=SVC(gamma='auto'))
classifier = RandomForestClassifier(n_estimators=1000)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)


cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

[[52  0]
 [ 0 28]]
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00        52
           1       1.00      1.00      1.00        28

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80



In [24]:
def evaluate_model(X_train, y_train, X_test, y_test, model):
	cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
	model.fit(X_train, y_train)
	return cross_val_score(model, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1)

models, names = list(), list()
# SVM
models.append(SVC(gamma='auto'))
names.append('SVM')
# KNN
models.append(KNeighborsClassifier())
names.append('KNN')
# Bagging
models.append(BaggingClassifier(n_estimators=1000))
names.append('BAG')
# RF
models.append(RandomForestClassifier(n_estimators=1000))
names.append(' RF')
# ET
models.append(ExtraTreesClassifier(n_estimators=1000))
names.append(' ET')

results = list()
for i in range(len(models)):
	scores = evaluate_model(X_train, y_train, X_test, y_test, models[i])
	results.append(scores)
	print('>%s %.3f (%.3f)' % (names[i], scores.mean(), scores.std()))

>SVM 0.829 (0.066)
>KNN 0.729 (0.075)
>BAG 1.000 (0.000)
> RF 0.988 (0.034)
> ET 0.992 (0.021)


In [11]:
"""
clf = RandomForestClassifier()
clf.fit(encoded_features, target)

plt.figure(figsize=(20,8))
plt.bar(encoded_features.columns, clf.feature_importances_)
plt.xticks(rotation=45)
"""

'\nclf = RandomForestClassifier()\nclf.fit(encoded_features, target)\n\nplt.figure(figsize=(20,8))\nplt.bar(encoded_features.columns, clf.feature_importances_)\nplt.xticks(rotation=45)\n'